<h1><center>FINAL ASSESSMENT</center></h1>          

## COURSE	NAME:   THE BIG DATA TOOLKIT
## COURSE	CODE:	MADSC103
## Name:                      Vidit Sharma 
## Student Code:        21041911

#### 1) Start a simple Spark Session

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Basics').getOrCreate()

#### 2) Load the Walmart Stock csv file, have Spark to infer the data types. 

In [3]:
from pyspark.sql.types import (StructField, StringType,
                               IntegerType, StructType)

data_schema = [StructField('Open', StringType(), True),
               StructField('High', StringType(), True),
               StructField('Low', StringType(), True),
               StructField('Close', StringType(), True),
               StructField('Volume', StringType(), True),
               StructField('Adj Close', StringType(), True)
              ]

final_struc = StructType(fields=data_schema)

df = spark.read.csv('walmart_stock.csv', inferSchema = True, header = True)

#### 3) How does the schema looks like?

In [4]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



##### 4)Print out the first 5 columns (hint: use a for loop)

In [5]:
for line in df.head(5):
    print(line, '\n')

Row(Date=datetime.datetime(2012, 1, 3, 0, 0), Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996) 

Row(Date=datetime.datetime(2012, 1, 4, 0, 0), Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475) 

Row(Date=datetime.datetime(2012, 1, 5, 0, 0), Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539) 

Row(Date=datetime.datetime(2012, 1, 6, 0, 0), Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922) 

Row(Date=datetime.datetime(2012, 1, 9, 0, 0), Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004) 



#### 5) Format the qualitative variables in your dataframe to show just two decimal places. (Open, High, Low, Close, Volume). 

Formatting all columns of the data frame to 2 decimal places

In [6]:
from pyspark.sql.functions import round
for c_name, c_type in df.dtypes:
    if c_type in ('double', 'float'):
        df = df.withColumn(c_name, round(c_name, 2))
df.show()

+-------------------+-----+-----+-----+-----+--------+---------+
|               Date| Open| High|  Low|Close|  Volume|Adj Close|
+-------------------+-----+-----+-----+-----+--------+---------+
|2012-01-03 00:00:00|59.97|61.06|59.87|60.33|12668800|    52.62|
|2012-01-04 00:00:00|60.21|60.35|59.47|59.71| 9593300|    52.08|
|2012-01-05 00:00:00|59.35|59.62|58.37|59.42|12768200|    51.83|
|2012-01-06 00:00:00|59.42|59.45|58.87| 59.0| 8069400|    51.46|
|2012-01-09 00:00:00|59.03|59.55|58.92|59.18| 6679300|    51.62|
|2012-01-10 00:00:00|59.43|59.71|58.98|59.04| 6907300|    51.49|
|2012-01-11 00:00:00|59.06|59.53|59.04| 59.4| 6365600|    51.81|
|2012-01-12 00:00:00|59.79| 60.0| 59.4| 59.5| 7236400|     51.9|
|2012-01-13 00:00:00|59.18|59.61|59.01|59.54| 7729300|    51.93|
|2012-01-17 00:00:00|59.87|60.11|59.52|59.85| 8500000|     52.2|
|2012-01-18 00:00:00|59.79|60.03|59.65|60.01| 5911400|    52.34|
|2012-01-19 00:00:00|59.93|60.73|59.75|60.61| 9234600|    52.86|
|2012-01-20 00:00:00|60.7

Displaying the summary (count, mean, SD, min and max) of the data frame columns using .describe() and formatting to 2 decimal places. 

In [7]:
from pyspark.sql.functions import format_number

summary = df.describe()
summary.select(summary['summary'],
              format_number(summary['Open'].cast('float'),2).alias('Open'),
              format_number(summary['High'].cast('float'),2).alias('High'),
              format_number(summary['Low'].cast('float'),2).alias('Low'),
              format_number(summary['Close'].cast('float'),2).alias('Close'),
              format_number(summary['Volume'].cast('float'),2).alias('Volume')).show()

+-------+--------+--------+--------+--------+-------------+
|summary|    Open|    High|     Low|   Close|       Volume|
+-------+--------+--------+--------+--------+-------------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|     1,258.00|
|   mean|   72.36|   72.84|   71.92|   72.39| 8,222,093.50|
| stddev|    6.77|    6.77|    6.74|    6.76| 4,519,781.00|
|    min|   56.39|   57.06|   56.30|   56.42| 2,094,900.00|
|    max|   90.80|   90.97|   89.25|   90.47|80,898,096.00|
+-------+--------+--------+--------+--------+-------------+



#### 6) Create a new dataframe with a column called HR ratio that is the ratio of the High Price versus volume of stock traded for a day. Change the format of HR ratio to show two decimal places. 

In [8]:
df_hr = df.withColumn('HR_Ratio', df['High']/df['Volume']).select(['HR_Ratio'])
df_hr.show()

+--------------------+
|            HR_Ratio|
+--------------------+
|4.819714574387472E-6|
|6.290848821573389...|
|4.669413073103491E-6|
|7.367338339901356E-6|
|8.915604928660188E-6|
|8.644477581688938E-6|
| 9.35182857861003E-6|
| 8.29141562102703E-6|
|7.712211972623653E-6|
|7.071764705882352...|
|1.015495483303447...|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448324825044...|
|8.316075414862431E-6|
|9.721183974042911E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows



Round off HR_Ratio to 2 decimal places

In [9]:
from pyspark.sql.functions import col
df_hr.withColumn("HR_Ratio", format_number(col("HR_Ratio"),2)).show(10,False)

+--------+
|HR_Ratio|
+--------+
|0.00    |
|0.00    |
|0.00    |
|0.00    |
|0.00    |
|0.00    |
|0.00    |
|0.00    |
|0.00    |
|0.00    |
+--------+
only showing top 10 rows



In the above output of rounding off we see that the result is 0.00 for all the values because the real number for 4.819714653321546E-6 is 0.000004819714653321546. 
However to see the number clearly we can round off to more than 5 decimal places as shown below. 

In [10]:
df_hr.withColumn("HR_Ratio", format_number(col("HR_Ratio"),8)).show(10,False)

+----------+
|HR_Ratio  |
+----------+
|0.00000482|
|0.00000629|
|0.00000467|
|0.00000737|
|0.00000892|
|0.00000864|
|0.00000935|
|0.00000829|
|0.00000771|
|0.00000707|
+----------+
only showing top 10 rows



#### 7) What is the mean of the Close column? Change the format to show two decimal places. 

In [11]:
from pyspark.sql.functions import mean

mean = df.select(mean('Close')).show()


+-----------------+
|       avg(Close)|
+-----------------+
|72.38844992050863|
+-----------------+



#### 8) What is the max and min of the Volume column? 

In [12]:
from pyspark.sql.functions import min, max
df.select(max('Volume'),min('Volume')).show()

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+



#### 9) How many days was the Close lower than 60 dollars?

In [13]:
df.filter(df['Close'] < 60).count()

81

#### 10) What percentage of the time was the High greater than 80 dollars? In other words, (Number of Days High >80) / (Total days in the dataset)

In [14]:
df.filter('High > 80').count() * 100/df.count()

9.141494435612083